# NDEx Client Tutorial v3.0


## Overview

The NDEx Python Client is a module that simplifies access to the NDEx Server API and provides convenience methods for common operations on networks.

In this tutorial, we will first show how to install the NDEx Python Client module and use it for basic operations on networks stored on an NDEx server.

### NDEx Client Objects

The NDEx Python Client provides an interface to an NDEx server that is managed via a client object class.  An NDEx Client object can be used to access an NDEx server either anonymously or using a specific user account. For each NDEx server and user account that you want to use in your script or application, you create an instance of the NDEx Client.

**This Tutorial Requires That You First Create a Personal Account on the NDEx Public Server**


## Requirements

The **NDEx Python Client 3.0** requires Python 2.7.9 and the latest version of the PIP Python package manager for installation.  [Click here](https://pypi.python.org/pypi/pip) to download the PIP Python package.


## Installing the NDEx Python Client Module

The NDEx Python Client is available via the ndex module in the Python Package Index (PyPI).  

This tutorial requires the 3.xxx release (or higher) of the ndex module. 

To install the ndex module to your Python environment using PIP:


```>>> pip install ndex``` 

To upgrade the ndex module to the latest version:

```>>> pip install ndex --upgrade```

The ndex and other modules may then be imported for this tutorial:

In [11]:
import ndex.client as nc
import io
import json
from IPython.display import HTML
from time import sleep

## Setting up NDEx Clients

In this section you will configure two client objects to access the public NDEx server. 

The first will enable you to make anonymous requests. 

The second will enable you to perform operations requiring authentication, such as saving networks to your account.

### Anonymous Clients

The following code creates an NDEx client object to access the NDEx public server anonymously, then tests the client by getting the current server status.

In [12]:
anon_ndex=nc.Ndex("http://public.ndexbio.org")
anon_ndex.update_status()
networks = anon_ndex.status.get("networkCount")
users = anon_ndex.status.get("userCount")
groups = anon_ndex.status.get("groupCount")
print("anon client: %s networks, %s users, %s groups" % (networks, users, groups))

anon client: 3097 networks, 672 users, 26 groups


### Personal Clients

A personal client enables you to perform operations requiring authentication, such as saving networks to your account.

You must first create an account on the NDEx Public Server to create a personal client object.

The following code creates an NDEx client object to access your account on the NDEx Public Server, then tests the client by getting the current server status.

For convenience and clarity in this example, we use the variables ‘my_account’ and ‘my_password’ to hold the strings for your account name and password. Edit the values for these variables before running the following cell.

In [13]:
my_account="drh"
my_password="drh"
my_ndex=nc.Ndex("http://public.ndexbio.org", my_account, my_password)
my_ndex.update_status()
networks = my_ndex.status.get("networkCount")
users = my_ndex.status.get("userCount")
groups = my_ndex.status.get("groupCount")
print("my_ndex client: %s networks, %s users, %s groups" % (networks, users, groups))

my_ndex client: 3097 networks, 672 users, 26 groups


## Working with the NDEx Network Using the Anonymous Client

### Get Network Information by Accession

You can access a network by its accession id, which is a universally unique identifier (UUID) assigned to the network by the NDEx server. All networks have a UUID and they are unique across all servers. No two networks will share an UUID.

In this step, you will get basic information about the network and retrieve a Network Summary structure. The ‘Metabolism of RNA’ network is in the NDEx Tutorials account on the public NDEx server; its UUID is ‘9ed0cd55-9ac0-11e4-9499-000c29202374’



anon_ndex will access the network using the **get_network_summary(network_id)** which returns a NetworkSummary as  
a dictionary:

In [14]:
ns=anon_ndex.get_network_summary('9ed0cd55-9ac0-11e4-9499-000c29202374')
def summary2table(object):
    table = "<table>"
    for key, value in object.items():
        if key == "warnings":
            warning_list = "" #<ul>"
            for warning in value:
                warning_list += "%s<br>" % warning
                #warning_list += "<li>%s</li>" % warning
            #warning_list = "</ul>"
            value = warning_list
        if key == "properties":
            property_table = "<table>"
            for property in value:
                property_table += "<tr>" 
                property_table += "<td>%s</td><td>%s</td>" % (property.get("predicateString"), property.get("value"))
                property_table += "</tr>"
            property_table += "</table>"
            value = property_table
                
        table += "<tr>" 
        table += "<td>%s</td><td>%s</td>" % (key, value)
        table += "</tr>"
    table += "</table>"
    return table

HTML(summary2table(ns))

### Find Networks by Search – Simple Search


You can search for networks by the text in their name and description as well as the names and controlled vocabulary terms associated with their nodes. The input is a search string that conforms to Lucene search string syntax, but in its simplest form is one or more search terms separated by spaces.

anon_ndex will perform the network search using the **search_networks(search_string="", account_name=None, start=0, size=100, include_groups=False)** method that returns a list of NetworkSummary dictionaries:


In [15]:
metabolic_networks=anon_ndex.search_networks('metabo*')
print("%s networks found." % (len(metabolic_networks['networks'])))
#print("\nNetworks:\n")
#for ns in metabolic_networks['networks']: print("  %s" % ns.get('name'))

68 networks found.


The search can also be limited to a specific account and to a number of search results:


In [16]:
metabolic_networks=anon_ndex.search_networks('metabo* userAdmin:ndextutorials', size=2)
print("%s networks found" % (len(metabolic_networks['networks'])))
print("\nNetworks:\n")
for ns in metabolic_networks['networks']: print("  %s" % ns.get('name'))

2 networks found

Networks:

  Metabolism
  Metabolism of proteins


### Get a Network


You can obtain an entire network as a CX stream, which is an NDEx format that is optimized for streaming networks. 
This is performed as a monolithic operation, so care should be taken when requesting very large networks. Applications
can use the **get_network_summary** method to check the node and edge counts for a network before attempting to use 
**get_complete_network**. The stream is contained in a Response object from the 
[Python requests library](http://docs.python-requests.org/en/master/).


anon_ndex will use the **get_complete_network(network_id)** method to get a Response object (from the [Python requests library](http://docs.python-requests.org/en/master/))

In [17]:
response=anon_ndex.get_network_as_cx_stream('9ed0cd55-9ac0-11e4-9499-000c29202374')
print("Received %s characters of CX" % len(response.content))

Received 665511 characters of CX


### Query a Network – Neighborhood Query


You can retrieve a ‘neighborhood’ subnetwork of a network as a CX stream. The query finds the subnetwork by first 
identifying nodes that are associated with identifiers in the search_string, then traversing a specified number of 
edges starting from those nodes.  The **search_depth** parameter controls the search, defaults to 1 edge and can be no more than 3 edges. 

The query to anon_ndex will use the **get_neighborhood_as_cx_stream(network_id, search_string, search_depth=1, edge_limit=2500)** method to get a Response object (from the [Python requests library](http://docs.python-requests.org/en/master/))

In [18]:
query_result_cx=anon_ndex.get_neighborhood('9ed0cd55-9ac0-11e4-9499-000c29202374', 'XRN1')

def getNumberOfNodesAndEdgesFromCX(cx):
    numberOfEdges = numberOfNodes = 0;
    for aspect in cx:
        if 'metaData' in aspect:
            metaData = aspect['metaData']
            for element in metaData:
                if (('name' in element) and (element['name'] == 'nodes')):
                    numberOfNodes = element['elementCount']
                if (('name' in element) and (element['name'] == 'edges')):
                    numberOfEdges = element['elementCount']
            break
    return numberOfNodes, numberOfEdges


nodes, edges = getNumberOfNodesAndEdgesFromCX(query_result_cx)

print("Query result network contains {0} nodes and {1} edges.".format(nodes,edges))


Query result network contains 20 nodes and 26 edges.


## Working with the NDEx Network Using Your Personal Client

### Create a Network

You can create a new network on an NDEx server if you have a CX stream. The network is created in the user account associated with the client object. All methods that create or modify content on the NDEx server require authentication, so you will use the my_ndex client object that you set up at the start of the tutorial and will create a network in your account.

In the previous section, your neighborhood query retrieved a small network which was bound to the variable **query_result_network**.

We will now save this network to your account using **save_cx_stream_as_new_network(network)** and receive the URI for the new network. The URI includes the network UUID.

In [19]:
uri = my_ndex.save_new_network(query_result_cx)
uuid = uri.rpartition('/')[-1]
print("URI of the newly created network {0} is {1}".format(uuid, uri))
for i in range(0, 4):
    sleep(1)
    new_summary = my_ndex.get_network_summary(uuid)
    if new_summary.get("isValid"):
        print("New network has been validated by the server.")
        break

URI of the newly created network 83998b7e-037e-11e7-aba2-0ac135e8bacf is http://public.ndexbio.org/v2/network/83998b7e-037e-11e7-aba2-0ac135e8bacf
New network has been validated by the server.


### Update Network Profile

With the network UUID, you can update the name, description and version of the new network using the method **update_network_profile(network_id, network_profile)**

In [20]:
#we use ID of the network we created at previous step
network_profile={"name":"Renamed Network", "description":"New Description", "version":"2.0"}
my_ndex.update_network_profile(uuid, network_profile)
new_summary = my_ndex.get_network_summary(uuid)
print("new name = %s" % new_summary.get('name'))
print("new description = %s" % new_summary.get('description'))
print("new version = %s" % new_summary.get('version'))

new name = Renamed Network
new description = New Description
new version = 2.0


### Set Read-Only

The new network can be set to **read-only** using the **set_read_only(network_id, boolean)** method, preventing unintended modification. 

In [21]:
#make network read-only
my_ndex.set_read_only(uuid, True)
new_summary = my_ndex.get_network_summary(uuid)
print("The read only status is %s" % new_summary.get('isReadOnly'))

The read only status is True


The network is then reverted to read-write, enabling modification.

In [22]:
#revert network to original state (make it read-write again)
my_ndex.set_read_only(uuid, False)
new_summary = my_ndex.get_network_summary(uuid)
print("The read only status has been reverted to %s" % new_summary.get('isReadOnly'))

The read only status has been reverted to False


### Delete a Network

Finally, the query result network is deleted using **delete_network(networkId)**.

Note that you can only delete networks that you own.

Be careful: there is no method to undo a deletion.

In [23]:
my_ndex.delete_network(uuid)
print("Tutorial Complete")

Tutorial Complete
